In [ ]:
import pandas as pd

# Load the hospital ratios pickle file
df = pd.read_pickle('../hospital_ratios.pkl')
df.head()

FY 2020  \
Hospital          Ratio                                                         
Bridgton Hospital Total Margin                                          6.940   
                  Operating Margin                                      4.040   
                  Non Operating Revenue                                43.490   
                  Return on Equity                                      8.050   
                  Current Ratio without Board Designated & Undesi...    3.106   

                                                                      FY 2021  \
Hospital          Ratio                                                         
Bridgton Hospital Total Margin                                          12.47   
                  Operating Margin                                       9.64   
                  Non Operating Revenue                                 25.11   
                  Return on Equity                                      33.80   
                  Current Ratio without Board Designated & Undesi...     1.43   

                                                                      FY 2022  \
Hospital          Ratio                                                         
Bridgton Hospital Total Margin                                         16.100   
                  Operating Margin                                     15.910   
                  Non Operating Revenue                                 1.370   
                  Return on Equity                                     78.360   
                  Current Ratio without Board Designated & Undesi...    0.792   

                                                                      FY 2023  \
Hospital          Ratio                                                         
Bridgton Hospital Total Margin                                         24.090   
                  Operating Margin                                     18.710   
                  Non Operating Revenue                                27.500   
                  Return on Equity                                     73.960   
                  Current Ratio without Board Designated & Undesi...    1.867   

                                                                      FY 2024  
Hospital          Ratio                                                        
Bridgton Hospital Total Margin                                           8.19  
                  Operating Margin                                       7.48  
                  Non Operating Revenue                                  9.44  
                  Return on Equity                                      20.94  
                  Current Ratio without Board Designated & Undesi...     1.20